In [1]:
import os
import ROOT
import numpy as np
import h5py
from tqdm import tqdm

Welcome to JupyROOT 6.30/02


In [2]:
input_path = "/store/hep/users/yewzzang/KNO/elec_500/"
output_path = "/store/cpnr/users/yewzzang/KNO_h5/pi0_100to1000/"

In [3]:
list_file = os.listdir(input_path)

In [4]:
for files in tqdm(range(len(list_file))):

    file_name = input_path + list_file[files]
    output_file = output_path + list_file[files][:-5] + '.h5'

    ROOT.gSystem.Load("/users/yewzzang/WCSim_KNO/WCSim_v1.8.0-build/libWCSimRoot.so")
    fin = ROOT.TFile(file_name)

    eventT = fin.Get("wcsimT")
    event = ROOT.WCSimRootEvent()
    eventT.SetBranchAddress("wcsimrootevent", event)
    eventT.GetBranch("wcsimrootevent").SetAutoDelete(1)
    eventT.GetEntry(0)
    nEvents = eventT.GetEntries()

    ## Load the geometry
    geomT = fin.Get("wcsimGeoT")
    geom = ROOT.WCSimRootGeom()
    geomT.SetBranchAddress("wcsimrootgeom", geom)
    geomT.GetEntry(0)
    nODPMTs = geom.GetODWCNumPMT()
    nPMTs = geom.GetWCNumPMT()

    print("--------------------")
    print(f" nEvents = {nEvents}")
    print(f" nPMTs   = {nPMTs}")
    print(f" nODPMTs = {nODPMTs}")
    print("--------------------")

    out_pmt_x = np.zeros(nPMTs)
    out_pmt_y = np.zeros(nPMTs)
    out_pmt_z = np.zeros(nPMTs)

    out_pmt_px = np.zeros(nPMTs)
    out_pmt_py = np.zeros(nPMTs)
    out_pmt_pz = np.zeros(nPMTs)

    for iPMT in range(nPMTs):
        pmt = geom.GetPMT(iPMT)
        out_pmt_x[iPMT] = pmt.GetPosition(0)
        out_pmt_y[iPMT] = pmt.GetPosition(1)
        out_pmt_z[iPMT] = pmt.GetPosition(2)
        out_pmt_px[iPMT] = pmt.GetOrientation(0)
        out_pmt_py[iPMT] = pmt.GetOrientation(1)
        out_pmt_pz[iPMT] = pmt.GetOrientation(2)

    print("@@@ Start analysing data")
    out_vtx_x = np.zeros(nEvents)
    out_vtx_y = np.zeros(nEvents)
    out_vtx_z = np.zeros(nEvents)
    out_vtx_t = np.zeros(nEvents)

    out_vtx_px = np.zeros(nEvents)
    out_vtx_py = np.zeros(nEvents)
    out_vtx_pz = np.zeros(nEvents)
    out_vtx_ke = np.zeros(nEvents)

    out_pmt_q = np.zeros((nEvents, nPMTs))
    out_pmt_t = np.zeros((nEvents, nPMTs))

    for iEvent in tqdm(range(nEvents)):
        eventT.GetEvent(iEvent)
        trigger = event.GetTrigger(0)

        nVtxs = trigger.GetNvtxs()
        if nVtxs < 1: continue
        out_vtx_x[iEvent] = trigger.GetVtx(0)
        out_vtx_y[iEvent] = trigger.GetVtx(1)
        out_vtx_z[iEvent] = trigger.GetVtx(2)
        out_vtx_t[iEvent] = 0

        out_vtx_px[iEvent] = 0
        out_vtx_py[iEvent] = 0
        out_vtx_pz[iEvent] = 0
        out_vtx_ke[iEvent] = 0

        nHitsC = trigger.GetNcherenkovdigihits()
        for iHit in range(nHitsC):
            hit = trigger.GetCherenkovDigiHits().At(iHit)
            iPMT = hit.GetTubeId()-1
            out_pmt_q[iEvent, iPMT] = hit.GetQ()
            out_pmt_t[iEvent, iPMT] = hit.GetT()

    kwargs = {'dtype':'f4', 'compression':'lzf'}
    with h5py.File(output_file, 'w', libver='latest') as fout:
        gGeom = fout.create_group('geom')
        gGeom.create_dataset('pmt_x', data=out_pmt_x, **kwargs)
        gGeom.create_dataset('pmt_y', data=out_pmt_y, **kwargs)
        gGeom.create_dataset('pmt_z', data=out_pmt_z, **kwargs)

        gGeom.create_dataset('pmt_px', data=out_pmt_px, **kwargs)
        gGeom.create_dataset('pmt_py', data=out_pmt_py, **kwargs)
        gGeom.create_dataset('pmt_pz', data=out_pmt_pz, **kwargs)

        gEvent = fout.create_group('event')
        gEvent.create_dataset('vtx_x', data=out_vtx_x, **kwargs)
        gEvent.create_dataset('vtx_y', data=out_vtx_y, **kwargs)
        gEvent.create_dataset('vtx_z', data=out_vtx_z, **kwargs)
        gEvent.create_dataset('vtx_t', data=out_vtx_t, **kwargs)

        gEvent.create_dataset('vtx_px', data=out_vtx_px, **kwargs)
        gEvent.create_dataset('vtx_py', data=out_vtx_py, **kwargs)
        gEvent.create_dataset('vtx_pz', data=out_vtx_pz, **kwargs)
        gEvent.create_dataset('vtx_ke', data=out_vtx_ke, **kwargs)

        gEvent.create_dataset('pmt_q', data=out_pmt_q, **kwargs)
        gEvent.create_dataset('pmt_t', data=out_pmt_t, **kwargs)

  0%|          | 0/500 [00:00<?, ?it/s]

--------------------
 nEvents = 2000
 nPMTs   = 30912
 nODPMTs = 0
--------------------
@@@ Start analysing data


  0%|          | 1/500 [00:18<2:31:24, 18.20s/it]

--------------------
 nEvents = 2000
 nPMTs   = 30912
 nODPMTs = 0
--------------------
@@@ Start analysing data


  0%|          | 2/500 [00:36<2:31:19, 18.23s/it]

...

100%|█████████▉| 498/500 [3:12:02<00:47, 23.59s/it]

--------------------
 nEvents = 2000
 nPMTs   = 30912
 nODPMTs = 0
--------------------
@@@ Start analysing data


100%|█████████▉| 499/500 [3:12:26<00:23, 23.50s/it]

--------------------
 nEvents = 2000
 nPMTs   = 30912
 nODPMTs = 0
--------------------
@@@ Start analysing data


100%|██████████| 500/500 [3:12:50<00:00, 23.14s/it]
Warning in <TClassTable::Add>: class WCSimRootCherenkovHit already in TClassTable
Warning in <TClassTable::Add>: class WCSimRootCherenkovDigiHit already in TClassTable
Warning in <TClassTable::Add>: class WCSimRootPi0 already in TClassTable
Warning in <TClassTable::Add>: class WCSimRootTrigger already in TClassTable
Warning in <TClassTable::Add>: class WCSimRootEvent already in TClassTable
Warning in <TClassTable::Add>: class WCSimRootGeom already in TClassTable
Warning in <TFile::Init>: no StreamerInfo found in /users/yewzzang/WCSim_KNO/WCSim_v1.8.0-build/libWCSimRootDict_rdict.pcm therefore preventing schema evolution when reading this file. The file was produced with version 6.28/04 of ROOT.
Error in <TUnixSystem::Load>: version mismatch, /users/yewzzang/WCSim_KNO/WCSim_v1.8.0-build/libWCSimRoot.so = 62804, ROOT = 63002
Warning in <TClassTable::Add>: class WCSimRootTrigger already in TClassTable
Warning in <TClassTable::Add>: class